## Basic set

In [1]:
import os, sys, random, struct, argparse
import numpy as np
np.set_printoptions(precision=16)

In [2]:
sys.path.remove('/home/jungeun/.local/lib/python3.6/site-packages')

In [3]:
import torch
sys.path.append('/home/jungeun/.local/lib/python3.6/site-packages')

In [4]:
parser = argparse.ArgumentParser(description='fixed_mac')
parser.add_argument('--full_bits', type=int, default=8, help='Number of Quantization Bits')
parser.add_argument('--frac_bits', type=int, default=4, help='Number of Quantization Bits')
parser.add_argument('--bBW', type=int, default=4, help='Number of bit width')
args = parser.parse_args(args=[])

## Integer -> Two's Complemenet Binary

In [5]:
def int2bin(iIn,iBW):
    iBW = iBW + 1
    if iIn >= 0:
        bOut = bin(iIn).replace('0b','').rjust(iBW,'0')
    else :
        bOut = bin(iIn & (pow(2,iBW)-1)).replace('0b','').rjust(iBW,'1')
    return bOut[1:]

## XOR gate

In [6]:
def XOR(iA,iB):
    if iA != iB :
        iOut = 1
    else :
        iOut = 0
    return iOut

In [7]:
def binInv(bIn):
    bOut = bin(int(bIn,2)^(pow(2,len(bIn))-1)).replace('0b','').rjust(len(bIn),'0')
    return bOut

In [8]:
2**-1

0.5

In [9]:
def FindFraction(bIn):
    s = 0
    for i in range(len(bIn)):
        if bIn[i] == '1' :
            s = s + 2**(-(i+1))
    return s

In [10]:
10111010
FindFraction('10111010')

0.7265625

## Fixed

In [11]:
class fxp:
    def __init__(self, bIn, iBWF):
        self.iFullBW = len(bIn)
        self.iIntgBW = self.iFullBW - iBWF
        self.bSign = bIn[0]
        self.bIntg = bIn[:self.iIntgBW]
        self.bFrac = bIn[self.iIntgBW:]
        self.fFull = 0
        try:
            for idx, bit in enumerate(bIn):
                if idx == 0:
                    self.fFull = self.fFull + int(bit,2) * -pow(2, self.iIntgBW - 1)
                else:
                    self.fFull = self.fFull + int(bit,2) * pow(2, self.iIntgBW - 1 - idx)
        except:
            print(bIn)
        self.dispFull = self.bIntg +"."+ self.bFrac 
        return

In [12]:
class flp2fix:
    def __init__(self, fIn, iBW, iBWF):
        self.fMin = - 2 ** (iBW - iBWF - 1)
        self.fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
        self.fResol		= 2 ** -iBWF
        if fIn < self.fMin or fIn > self.fMax:
            print(f'({fIn}): Out of input range ({self.fMax}/{self.fMin}) during flp -> fix converting ')
        self.iBW = iBW
        self.iBWI = iBW - iBWF
        self.iBWF = iBWF

        self.iFLP2INT = abs(int(fIn * 2 ** iBWF))
        if fIn < 0:
            self.iFLP2INT = 2 ** (iBW-1) - self.iFLP2INT

        if fIn >= 0:
            self.bFull = bin(self.iFLP2INT)[2:].rjust(iBW, '0')
        else:
            self.bFull = '1'+bin(self.iFLP2INT)[2:].rjust(iBW-1, '0')
            if len(self.bFull) > iBW:
                self.bFull = '0' * iBW

        self.cssFxp = fxp(self.bFull, self.iBWF)
        self.bSign = self.cssFxp.bSign
        self.bIntg = self.cssFxp.bIntg
        self.bFrac = self.cssFxp.bFrac
        self.fFull = self.cssFxp.fFull
        return

In [13]:
class LFSR4:
    def Random(self):
        self.b0 = eval(f'str(random.randint(0,1))')
        self.b1 = eval(f'str(random.randint(0,1))')
        self.b2 = eval(f'str(random.randint(0,1))')
        self.b3 = eval(f'str(random.randint(0,1))')
        if int(self.b0) + int(self.b1) + int(self.b2) + int(self.b3)  == 0 :
            self.b0 = eval(f'str(random.randint(0,1))')
            self.b1 = eval(f'str(random.randint(0,1))')
            self.b2 = eval(f'str(random.randint(0,1))')
            self.b3 = eval(f'str(random.randint(0,1))')
        return self.b0 + self.b1 + self.b2 + self.b3 
    
    def Normal(self,stream):
        self.b0 = str(XOR(int(stream[2]),int(stream[3])))
        self.b1 = str(stream[0])
        self.b2 = str(stream[1])
        self.b3 = str(stream[2])
        
        return self.b0 + self.b1 + self.b2 + self.b3 
    
    def Allzero(self):
        self.b0 = '0'
        self.b1 = '0'
        self.b2 = '0'
        self.b3 = '0'
        
        return self.b0 + self.b1 + self.b2 + self.b3 

In [14]:
def LFSRlist4():
    lfsr = LFSR4()
    lfsrlist = []
    for k in range(2**(args.bBW)-1): #lfsr number generating
        if k == 0:
            lfsrlist.append(lfsr.Random())
        else :
            lfsrlist.append(lfsr.Normal(lfsrlist[k-1]))
        if (k == 2**(args.bBW)-2):
            lfsrlist.append(lfsr.Allzero())
    
    if (args.bBW) != args.frac_bits :
        if args.bBW < args.frac_bits :
            for i in range(len(lfsrlist)):
                lfsrlist[i] = lfsrlist[i] + (args.frac_bits-args.bBW)*'0'
        else :
            print("it can't work")
            return 0
    
    return lfsrlist

## Comparator

In [84]:
def Comp(a,lfsr,snum): #lfsr number < origin number , output = 1
    for com in range(0,len(a)):
        oA = 0
        if a[com]!=lfsr[com]:
            if(int(a[com]) > int(lfsr[com])):
                oA = 1
            else :
                oA = 0
            break
    return XOR(oA,snum)

## SNUM

In [85]:
def snum(a):
    if a >= 0 :
        return 0
    else :
        return 1

## SNG module

In [86]:
def SNG(iIN,lfsr):
    if iIN == 1:
        iIN = 0.9999
    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    bFRAC = bIN[-(args.frac_bits):]
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating
        lNUM = lfsr[k]
        a = Comp(bFRAC,lNUM,sNUM)
        oAlist.append(a) #comparator of input a
    
    oAlist.insert(0,sNUM)
    #sA = "".join(oAlist)
    if bIN == args.full_bits*'0' :
        return '0'*(2**(args.bBW)+1)
    else :
        return torch.tensor(oAlist)

In [87]:
def CountOne1(nIn):
    n = 0
    for num in nIn:
        if num == 1 :
            n += 1
    if nIn[0] == 1 :
        return n-1
    else :
        return n

In [88]:
def defSign1(nIn):
    if nIn[0] == 1 :
        return -1
    else :
        return 1

In [89]:
def S2None(sIn):
    s = defSign1(sIn)
    n = CountOne1(sIn)
    o = (n/(2**args.bBW))*s
    return o

In [90]:
a = -random.random()
b = flp2fix(a,args.full_bits,args.frac_bits).fFull
print(b)
lfsr = LFSRlist4()
e = SNG(b,lfsr)
f = S2None(e)

print(f)

-0.125
-0.125


In [91]:
for i in range(100):
    a = -random.random()
    b = flp2fix(a,args.full_bits,args.frac_bits).fFull
    c = flp2fix(a,args.full_bits,args.frac_bits).bFull
    d = flp2fix(b,args.full_bits,args.frac_bits).fFull
    lfsr = LFSRlist4()
    
    e = SNG(b,lfsr)
    print(b)
    for n in range(len(e)) :
        print(e[n].item(), end = '')
    f = S2None(e)
    print('')
    print(f)
    
    


-0.3125
10000010100111000
-0.3125
-0.8125
10110111111111110
-0.8125
-0.6875
11111111110010010
-0.6875
-0.9375
11111111111111110
-0.9375
-0.5625
10010011010111110
-0.5625
-0.8125
11110110111111110
-0.8125
-0.5
10101111000100110
-0.5
-0.1875
10000000011100000
-0.1875
-0.625
11110111110010010
-0.625
-0.9375
11111111111111110
-0.9375
-0.4375
11100000011010110
-0.4375
-0.0625
10010000000000000
-0.0625
-0.3125
10000001010011100
-0.3125
-0.75
11111100110111110
-0.75
-0.5625
11011111001001100
-0.5625
-0.6875
11001111111111000
-0.6875
0.0


AttributeError: 'str' object has no attribute 'item'